- Fragmentation (from [here](https://dl.acm.org/citation.cfm?id=2893731))
- $\text{fragmentation}(x) = 1 - \frac{x \times\#(\text{free chuncks of size } x)}{\text{free space}}$

In [1]:
%matplotlib ipympl
import random

https://github.com/matplotlib/jupyter-matplotlib

In [2]:
def add_var(alloc_size=30, alloc_time=1, dealloc_time=10):
    return {"alloc_size":alloc_size, "alloc_time":alloc_time, "dealloc_time":dealloc_time}

In [3]:

mpattern = []

mpattern.append(add_var(1,2,3))
mpattern.append(add_var(1,2,3))
mpattern.append(add_var(1,2,3))

In [4]:
mpattern

[{'alloc_size': 1, 'alloc_time': 2, 'dealloc_time': 3},
 {'alloc_size': 1, 'alloc_time': 2, 'dealloc_time': 3},
 {'alloc_size': 1, 'alloc_time': 2, 'dealloc_time': 3}]

In [13]:
class mpattern:
    'Allocation pattern'
    allocation_list = [] # allocation_list is a list of byte sizes of objects to be allocated.
    size = 0 # memory size in bytes
    
    def __init__(self, size):
        self.allocation_list = []
        self.size = size
        self.free_mem = size
        self.memory = [ False ] * size
        
    def add_var(self, alloc_size=30, alloc_time=0, dealloc_time=1):
        self.allocation_list.append({"alloc_size":alloc_size, "alloc_time":alloc_time, "dealloc_time":dealloc_time})
        self.free_mem -= alloc_size
        
    def allocate(self, size):
        allocated = False
        while not allocated:
            pick_place_to_allocate = random.randint(0,self.size-1-size)
            if self.isThereSpace(pick_place_to_allocate, size):
                self.writeAllocation(pick_place_to_allocate, size)
                allocated = True
                
    def writeAllocation(self, position, size):
        self.memory[position: position+size] = [True] * size
            
    def isThereSpace(self, position, size):
        if position+size > self.size:
            return 0
        return sum(self.memory[position: position+size]) == 0
            
    def __str__(self):
        str1 = "size: {}\n".format(str(self.size))
        for var in self.allocation_list:
            #str1 += str(var) + "\n"
            str1 += "alloc_size: " + str(var["alloc_size"]) + "\n"
        str1 += str(list(int(i) for i in self.memory))
        return str1
    
    def create_random(self, size, number):
        assert(False)
        
    def get_number_free_chunks(self, size):
        num = 0
        i = 0
        while i < self.size:
            if self.isThereSpace(i,size):
                num += 1
                i += size
            else:
                i += 1
        print(num)
        return num
        
    def get_fragmentation(self, size_x):
        if size_x > self.free_mem:
            return 1
        num_free_chunks_of_size_x = self.get_number_free_chunks(size_x)
        return 1-size_x*num_free_chunks_of_size_x/self.free_mem

In [14]:
mem = mpattern(200)

In [15]:
#mem.add_var(1)

In [16]:
mem.allocate(3)

In [17]:
print(mem)

size: 200
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]


In [18]:
mem.get_fragmentation(30)

6


0.09999999999999998

Some examples:

In [19]:
import matplotlib.pyplot as plt
plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt


plt.rcdefaults()
fig, ax = plt.subplots()

# Example data
people = ('Tom', 'Dick', 'Harry', 'Slim', 'Jim')
y_pos = np.arange(len(people))
performance = 3 + 10 * np.random.rand(len(people))
error = np.random.rand(len(people))

ax.barh(y_pos, performance, xerr=error, align='center',
        color='green', ecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(people)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Performance')
ax.set_title('How fast do you want to go today?')

plt.show()

FigureCanvasNbAgg()

In [20]:
import matplotlib.pyplot as plt
plt.figure()
x = [1,2,3]
y = [4,5,6]
plt.plot(x,y)
plt.show()

FigureCanvasNbAgg()